### 실무에 적용해서 사용해보자
1. 교재에는 없는 내용으로 인터넷에서 보고 함. 교재사이트
2. GPT(BERT와 마찬가지로 사전훈련된 모델)를 이용해서 문장생성
3. 실무에 적용해도 정말 손색이 없다.

In [1]:
import numpy as np
import random
import tensorflow as tf
from transformers import AutoTokenizer
from transformers import TFGPT2LMHeadModel

c:\Users\user06\anaconda3\Lib\site-packages\keras\src\export\tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


TFGPT2LMHeadModel.from_pretrained('GPT 모델 이름')을 넣으면 두 개의 문장이 이어지는 문장 관계인지 여부를 판단하는 GPT 구조를 로드합니다. AutoTokenizer.from_pretrained('모델 이름')을 넣으면 해당 모델이 학습되었을 당시에 사용되었던 토크나이저를 로드합니다.

In [2]:
model = TFGPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2', from_pt=True)
tokenizer = AutoTokenizer.from_pretrained('skt/kogpt2-base-v2')


TensorFlow and JAX classes are deprecated and will be removed in Transformers v5. We recommend migrating to PyTorch classes or pinning your version of Transformers.
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2LMHeadModel: ['transformer.h.7.attn.masked_bias', 'transformer.h.1.attn.masked_bias', 'transformer.h.8.attn.masked_bias', 'transformer.h.6.attn.masked_bias', 'transformer.h.4.attn.masked_bias', 'transformer.h.0.attn.masked_bias', 'transformer.h.5.attn.masked_bias', 'transformer.h.2.attn.masked_bias', 'transformer.h.10.attn.masked_bias', 'transformer.h.9.attn.masked_bias', 'transformer.h.11.attn.masked_bias', 'transformer.h.3.attn.masked_bias', 'lm_head.weight']
- This IS expected if you are initializing TFGPT2LMHeadModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFGPT

GPT의 입력으로는 정수 인코딩 된 결과가 입력되어야 하므로 tokenizer.encode()를 통해서 '근육이 커지기 위해서는'이라는 문자열을 정수 시퀀스로 변환해줍니다.

In [3]:
sent = '근육이 커지기 위해서는'
input_ids = tokenizer.encode(sent)
input_ids = tf.convert_to_tensor([input_ids])
print(input_ids)

tf.Tensor([[33245 10114 12748 11357]], shape=(1, 4), dtype=int32)


33245 10114 12748 11357라는 5개의 정수 시퀀스를 얻습니다. 해당 정수 시퀀스를 GPT의 입력으로 사용하여 GPT가 이어서 문장을 생성하도록 해봅시다. 주어진 문장으로부터 이어서 문장을 생성하도록 하는 것은 model.generate()를 사용합니다.

In [4]:
# 이렇게 하면 기존의 정수 인코딩에 추가로 이어질 문장의 인코딩이 붙는다.
output = model.generate(input_ids,
                        max_length=128,
                        repetition_penalty=2.0,
                        use_cache=True)
print(output)
output_ids = output.numpy().tolist()[0]
print(output_ids)


tf.Tensor(
[[33245 10114 12748 11357 23879 39306  9684  7884 10211 15177 26421   387
  17339  7889  9908 15768  6903 15386  8146 12923  9228 18651 42600  9564
  17764  9033  9199 14441  7335  8704 12557 32030  9510 18595  9025 10571
  25741 10599 13229  9508  7965  8425 33102  9122 21240  9801 32106 13579
  12442 13235 19430  8022 12972  9566 11178  9554 24873  7198  9391 12486
   8711  9346  7071 36736  9693 12006  9038 10279 36122  9960  8405 10826
  18988 25998  9292  7671  9465  7489  9277 10137  9677  9248  9912 12834
  11488 13417  7407  8428  8137  9430 14222 11356 10061  9885 19265  9377
  20305  7991  9178  9648  9133 10021 10138 30315 21833  9362  9301  9685
  11584  9447 42129 10124  7532 17932 47123 37544  9355 15632  9124 10536
  13530 12204  9184 36152  9673  9788  9029 11764]], shape=(1, 128), dtype=int32)
[33245, 10114, 12748, 11357, 23879, 39306, 9684, 7884, 10211, 15177, 26421, 387, 17339, 7889, 9908, 15768, 6903, 15386, 8146, 12923, 9228, 18651, 42600, 9564, 17764, 9

기존의 33245 10114 12748 11357 라는 5개의 정수 시퀀스 뒤에도 여러 정수들이 추가로 생성된 것을 볼 수 있습니다. 정수들이 단순히 나열된 것만으로는 GPT가 실제로 어떤 문장을 생성했는지 알기 어려우니 해당 정수 시퀀스를 한국어로 변환해봅시다. 이 과정은 tokenizer.decode()를 사용하여 가능합니다.

In [5]:
tokenizer.decode(output_ids)

'근육이 커지기 위해서는 무엇보다 규칙적인 생활습관이 중요하다.\n특히, 아침식사는 단백질과 비타민이 풍부한 과일과 채소를 많이 섭취하는 것이 좋다.\n또한 하루 30분 이상 충분한 수면을 취하는 것도 도움이 된다.\n아침 식사를 거르지 않고 규칙적으로 운동을 하면 혈액순환에 도움을 줄 뿐만 아니라 신진대사를 촉진해 체내 노폐물을 배출하고 혈압을 낮춰준다.\n운동은 하루에 10분 정도만 하는 게 좋으며 운동 후에는 반드시 스트레칭을 통해 근육량을 늘리고 유연성을 높여야 한다.\n운동 후 바로 잠자리에 드는 것은 피해야 하며 특히 아침에 일어나면 몸이 피곤해지기 때문에 무리하게 움직이면 오히려 역효과가 날 수도 있다.\n운동을'

위에서는 다음 단어들로 올 여러개중 탑1만 보여준것임. 근육이 커지기 위해서는 "무엇보다" 첫번째 였다.
상위 5개의 단어를 뽑아보자

In [6]:
output = model(input_ids)
print(output)
# 아래코드는 첫번째 항목의 맨마지막 항목을 의미한단 -1 이니까
top5 = tf.math.top_k(output.logits[0, -1], k=5)

# 아래 2개를 찍어본 이유는 형태가 어떤지 볼려구
# print(output.logits[0, -1]). # 첫번째의 마지막행에 가장 유사한 놈들이 오는 것 같음. shape(1,4,51200) 구조를 가짐
# print(top5)

TFCausalLMOutputWithCrossAttentions(loss=None, logits=<tf.Tensor: shape=(1, 4, 51200), dtype=float32, numpy=
array([[[-5.2968407 , -5.245325  , -4.7947307 , ...,  0.02459955,
         -1.1457446 , -0.64933807],
        [-4.790766  , -5.411332  , -4.8759823 , ..., -0.93104184,
         -3.6518135 , -5.818618  ],
        [-4.0485926 , -3.5684733 , -5.736005  , ..., -2.4099343 ,
         -2.4635649 , -3.5291762 ],
        [-5.4558253 , -6.024943  , -6.139881  , ..., -0.22451176,
         -4.02617   , -3.2468672 ]]], shape=(1, 4, 51200), dtype=float32)>, past_key_values=(<tf.Tensor: shape=(2, 1, 12, 4, 64), dtype=float32, numpy=
array([[[[[-8.03373396e-01,  8.01336169e-01, -4.45047349e-01, ...,
           -7.34241843e-01, -5.77974796e-01, -6.26563847e-01],
          [-7.67724872e-01,  2.05992818e+00, -1.59064341e+00, ...,
           -1.66879833e+00, -1.80058050e+00, -3.24596226e-01],
          [-1.14716090e-01,  2.45837069e+00, -7.61260450e-01, ...,
           -1.04049337e+00, -5.28934240e

In [7]:
tokenizer.convert_ids_to_tokens(top5.indices.numpy())

['▁무엇보다', '▁우선', '▁반드시', '▁피부', '▁무엇보다도']

앞서 문장을 생성했을 당시에는 각 시점(time step)마다 가장 확률이 높은 단어를 예측했지만, 이번에는 매 시점마다 Top 5개의 단어들 중에서 랜덤으로 선택하는 방식을 택하여 문장을 생성해봅시다.

In [8]:
sent = '근육이 커지기 위해서는'
input_ids = tokenizer.encode(sent)

while len(input_ids) < 50:

    # model 사용법이다. 암기. 행렬을 넣는다.
    output = model(np.array([input_ids]))

    # Top 5의 단어들을 추출
    top5 = tf.math.top_k(output.logits[0, -1], k=5)

    # Top 5의 단어들 중 랜덤으로 다음 단어로 선택.
    token_id = random.choice(top5.indices.numpy())
    input_ids.append(token_id)

tokenizer.decode(input_ids)


"근육이 커지기 위해서는 무엇보다도 자신의 체중을 줄이는 것뿐이다”며 “아침 저녁 식사 후 체중을 줄이려다 갑자기 체중에 문제가 생긴 경우에는 평소보다 체중의 양을 늘리는 운동을 해야한다”고도 말했다고 전했다. 롯데의 '롯"